In [5]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [122]:
from proj1_helpers import *
#from implementations import *
from Leo import *
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [155]:
#print(tX[:,22])
# found that the jet number is in column 22

# isolate the jet numbers and remove jet number column:
x_j0 = tX[np.where(tX[:, 22] == 0)]
x_j0 = np.hstack((x_j0[:,:22], x_j0[:,23:]))

x_j1 = tX[np.where(tX[:, 22] == 1)]
x_j1 = np.hstack((x_j1[:,:22], x_j1[:,23:]))

x_j2 = tX[np.where(tX[:, 22] == 2)]
x_j2 = np.hstack((x_j2[:,:22], x_j2[:,23:]))

x_j3 = tX[np.where(tX[:, 22] == 3)]
x_j3 = np.hstack((x_j3[:,:22], x_j3[:,23:]))

# remove the jet column from the original data set
tX_no_jet = np.hstack((tX[:,:22], tX[:,23:]))

# append the y and ids columns such that they are treated as well
N = len(y)
tX_full = np.hstack((tX_no_jet, np.reshape(y, (N, 1)), np.reshape(ids, (N, 1))))

def column_remover_999(x, verbose = True):
    col_num_initial = len(x[0,:])
    x = x.transpose()
    lost_cols = np.where(np.all(x == -999, 1))[0]
    x = x[np.where(np.any(x != -999, 1))]
    x = x.transpose()
    if verbose:
        print("Lost ", col_num_initial - len(x[0,:]), " columns. The lost columns are: ", lost_cols)
    
    return x

x_j0 = column_remover_999(x_j0, verbose = False) # loses 10 columns:  4  5  6 12 22 23 24 25 26 27
x_j1 = column_remover_999(x_j1, verbose = False) # loses 7 columns:  4  5  6 12 25 26 27
x_j2 = column_remover_999(x_j2, verbose = False) # loses no columns
x_j3 = column_remover_999(x_j3, verbose = False) # loses no columns

def row_cleaner_999(x, verbose = True):
    len_initial = len(x)
    x = x[np.where(np.all(x != -999, 1))]
    if verbose:
        print("Lost ", 100 * (1 - len(x) / len_initial), " % of your data by removing ", len_initial - len(x), " rows")
    
    return x

x_j0 = cleaner_999(x_j0, verbose = False) # loses 26.1 % of data
x_j1 = cleaner_999(x_j1, verbose = False) # loses 9.8 % of data
x_j2 = cleaner_999(x_j2, verbose = False) # loses 5.6 % of data
x_j3 = cleaner_999(x_j3, verbose = False) # loses 6.7 % of data

def clean_999(x, verbose = True):
    #bad_col_set = [ 4,  5,  6, 12, 22, 23, 24, 25, 26, 27]
    x = np.hstack((x[:,:4], x[:,7:12], x[:,13:22], x[:,28:]))
    x = row_cleaner_999(x, verbose)
    return x

tX_no_999 = clean_999(tX_full) # Loses a total of 15.2% of the data

# Note: discuss the extent to which it makes sense to discuss outliers for a classification problem
def remove_outliers_kIQR(x, k = 3, verbose = True):
    len_initial = len(x)
    Q1, Q3 = np.quantile(x, [0.25, 0.75], 0)
    IQR = Q3 - Q1
    x = x[np.where((np.all(x > Q1 - k * IQR, 1)) & (np.all(x < Q3 + k * IQR, 1)))] # Tukey's Fence to filter points that are "far out"
    len_new = len(x)
    if verbose:
        print("Lost ", 100 * (1 - len_new / len_initial), " % of your data by detecting ", len_initial - len_new, " outliers")

    return x


tX_clean = remove_outliers_kIQR(tX_no_999, 3)

new_tX, new_y, new_ids = tX_clean[:, :-2], tX_clean[:, -2], tX_clean[:, -1]


# Store tX, with y in the second last column and the ids in the last column
with open('Leo_clean_dat.txt', 'w') as f:
    for row in tX_clean:
        for item in row:
            f.write("%s," % item)
        f.write("\n")
    print("file_written")

f.close()

Lost  15.245600000000003  % of your data by removing  38114  rows
Lost  8.554128163257602  % of your data by detecting  18125  outliers
file_written


In [154]:
w0 = np.ones(np.shape(new_tX)[1])*0
print("hi")
w, loss = least_squares_SGD(new_y, new_tX, w0, 20, 0.8)
print(w)
print(loss)

hi
[-1.69943214e+91 -9.09508239e+90 -1.36319924e+91 -2.29124144e+90
 -3.72433326e+89 -1.01991642e+90 -1.22164072e+91 -2.34751146e+89
  1.39060725e+89 -2.67055703e+90 -3.30423210e+88 -1.52731033e+89
 -6.37407764e+90  1.80761260e+89  1.52138559e+89 -3.63070949e+90
 -2.22163469e+89 -2.93795541e+91 -3.17177252e+90]
7.5562744411050735e+177


In [43]:
w, loss = least_squares(y, tX)
print(shape(tX))
print(w)
print(loss)

/home/machine-learning-higgs/scripts/implementations.py:36: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w = np.linalg.lstsq(tx,y)[0]


(250000, 30)
[ 8.03494351e-05 -7.20202266e-03 -6.05417274e-03 -5.47559078e-04
 -1.93874687e-02  4.73451612e-04 -2.60379058e-02  3.25106299e-01
 -3.80780004e-05 -2.72787427e+00 -2.21220141e-01  9.50794097e-02
  6.40351607e-02  2.73613395e+00 -3.31801099e-04 -9.54325141e-04
  2.74089070e+00 -5.34165277e-04  9.73498906e-04  3.69225050e-03
  3.54487171e-04 -5.43344618e-04 -3.30448035e-01 -1.40800497e-03
  8.31432843e-04  1.02117275e-03 -1.68047418e-03 -5.83664781e-03
 -1.11088003e-02  2.72833421e+00]
0.3396868094770344


## Generate predictions and save ouput in csv format for submission:

In [4]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

OSError:  not found.

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)